# *** Notebook stricly used for debugging purposes ***

In [38]:
# %load_ext autoreload
# %autoreload 2
# %reload_ext autoreload
from julia_import import Extremes
from structures.conversions import py_list_to_jl_vector, py_variable_to_jl_variable
from structures.dataitem import Variable

from xhydro.extreme_value_analysis.structures.conversions import (
    jl_vector_tuple_to_py_list,
)
from xhydro.extreme_value_analysis.structures.util import jl_variable_fit_parameters

In [39]:
def gevfit_debug(
    y: list[float],
    locationcov: list[Variable] = [],
    logscalecov: list[Variable] = [],
    shapecov: list[Variable] = [],
):
    jl_y = py_list_to_jl_vector(y)
    jl_locationcov, jl_logscalecov, jl_shapecov = jl_variable_fit_parameters(
        [locationcov, logscalecov, shapecov]
    )
    jl_fm = Extremes.gevfit(
        jl_y,
        locationcov=jl_locationcov,
        logscalecov=jl_logscalecov,
        shapecov=jl_shapecov,
    )
    params = jl_vector_tuple_to_py_list(Extremes.params(jl_fm))
    return {"params": params, "jl_fm": jl_fm}

In [40]:
y = [
    31.4,
    38.8,
    63.78,
    54.82,
    123.2,
    108.2,
    108.1,
    98.52,
    42.24,
    33.45,
    60.0,
    84.23,
    48.07,
    77.71,
    38.37,
    42.02,
    46.0,
]
# jl_y = py_list_to_jl_vector(y)
# cool_variable = Variable("cool_variable", y)
# jl_cool_variable = py_variable_to_jl_variable(cool_variable)
# blockmaxima = Extremes.seval('BlockMaxima{Distributions.GeneralizedExtremeValue}')(jl_cool_variable, None, None, None)

In [41]:
fit = gevfit_debug(y)
fit["params"]

[46.987584008046504, 16.492506293527303, 0.445392084357781]

In [42]:
fit["jl_fm"]

MaximumLikelihoodAbstractExtremeValueModel
model :
	BlockMaxima{GeneralizedExtremeValue}
	data :		Vector{Real}[17]
	location :	μ ~ 1
	logscale :	ϕ ~ 1
	shape :		ξ ~ 1

θ̂  :	[46.987584008046504, 2.8029061137141675, 0.445392084357781]


In [43]:
r = Extremes.returnlevel(fit["jl_fm"], 10)
r.value

1-element Vector{Float64}:
 110.84551168657379

In [58]:
import pandas as pd
import pooch

# Dataset taken from tests in Extremes.jl
GITHUB_URL = "https://github.com/hydrologie/xhydro-testdata"
BRANCH_OR_COMMIT_HASH = "extreme_value_analysis"

genextreme_data = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/extreme_value_analysis/genextreme.zip",
    known_hash="md5:cc2ff7c93949673a6acf00c7c2fac20b",
    processor=pooch.Unzip(),
)
GEV_STATIONARY = pd.read_csv(genextreme_data[0])
GEV_NONSTATIONARY = pd.read_csv(genextreme_data[1])


genpareto_data = pooch.retrieve(
    url=f"{GITHUB_URL}/raw/{BRANCH_OR_COMMIT_HASH}/data/extreme_value_analysis/genpareto.zip",
    known_hash="md5:ecb74164db4bbfeabfc5e340b11e7ae8",
    processor=pooch.Unzip(),
)
GP_STATIONARY = pd.read_csv(genpareto_data[0])
GP_NONSTATIONARY = pd.read_csv(genpareto_data[1])


y = GP_NONSTATIONARY["y"].values
y

array([0.50708462, 0.86535667, 0.57872479, ..., 2.84709803, 3.42667611,
       0.0619862 ])